In [ ]:
import glob
import os
import csv
import numpy as np
    
def information_gain(angles,bank_particles,weights):
    """
    Takes in angles as [theta,phi] and bank_particles and weigt as how they are given by the file. 
    """
    angle_dict={
        "theta":angles[0],
        "phi": angles[1]
    }
    best_guess=np.array(np.einsum('i...,i',bank_particles,weights))
    return adaptive_cost_func(angle_dict,bank_particles,weights,best_guess)       

def adaptive_cost_func(angles,rhoBank,weights,bestGuess):
    """
    Computes the expected entropy reduction of the posterior (likelihood) distribution.
    The angles are taken in as a dictionary and indicate what mesaurement is to be perfomred.
    Noise correction is currently removed.  
    """
    # Crates projector from angles
    #meshState=angles_to_state_vector(angles)
    
    #out=np.einsum('ij,ik->ijk',meshState,meshState.conj())
    povm=angles_to_single_qubit_POVM(angles)
    # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
    K=Shannon_entropy(np.einsum('ijk,kj->i',povm,bestGuess))
    J=Shannon_entropy(np.einsum('ijk,lkj->il',povm,rhoBank))
    # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
    return np.real(K-np.dot(J,weights))

def Shannon_entropy(prob):
    """
    Returns the shannon entorpy of the probability histogram. 
    """
    return np.real(np.sum(-(prob*np.log2(prob)),axis=0))

def angles_to_single_qubit_POVM(angles):
    """
    Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array . 
    For single qubit only.
    """
    up_state_vector=np.array([np.cos(angles["theta"]/2),np.exp(1j*angles["phi"])*np.sin(angles["theta"]/2)],dtype=complex)
    up_POVM=np.einsum("i,j->ij",up_state_vector,up_state_vector.conj())
    return np.array([up_POVM,np.eye(2)-up_POVM],dtype=complex)


#print(data_list[2][1])

path= "SL_data/"
cvs_names=glob.glob(f'{path}*.csv')
bank_names=glob.glob(f'{path}*bank.npy')
weight_names=glob.glob(f'{path}*weights.npy')

# Example of loop that load from all file names
# for names in cvs_names:
#     with open(f'{names}','r') as f:
#         csvreader =csv.reader(f,delimiter=",")
#         data_list=list(csvreader)
#     #print(data_list)
# for name in bank_names:
#     bank=np.load(name,mmap_mode="r")
#     #print(bank)
# for name in weight_names:
#     weights=np.load(name,mmap_mode="r")
#     #print(weights)

# Select a single file
ind=0
with open(f'{cvs_names[ind]}','r') as f:
    csvreader =csv.reader(f,delimiter=",")
    data_list=list(csvreader)
bank=np.load(bank_names[ind],mmap_mode="r")  
weights=np.load(weight_names[ind],mmap_mode="r")
        
NN_angle_pred=np.array([1,0.5]) # Format as [ theta, phi ]


# Select which step in ABME to predict
prediction_index=50

true_angle=np.array([float(data_list[prediction_index][3]),float(data_list[prediction_index][4])])
true_info_gain=information_gain(true_angle,bank[prediction_index],weights[prediction_index])
NN_pred_info_gain=information_gain(NN_angle_pred,bank[prediction_index],weights[prediction_index])
print(f'Info gain difference: {true_info_gain-NN_pred_info_gain}')


# Test to see that step 10 to 60 works
for i in range(50):
    prediction_index=10 +i
    true_angle=np.array([float(data_list[prediction_index][3]),float(data_list[prediction_index][4])])
    true_info_gain=information_gain(true_angle,bank[prediction_index],weights[prediction_index])
    NN_pred_info_gain=information_gain(NN_angle_pred,bank[prediction_index],weights[prediction_index])
    #print(true_info_gain)
    #print(NN_pred_info_gain)
    print(true_info_gain-NN_pred_info_gain)